In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!ls

In [2]:
!pip install tensorflow==1.10.0

    100% |████████████████████████████████| 58.4MB 645kB/s 
    100% |████████████████████████████████| 573kB 15.7MB/s 
    100% |████████████████████████████████| 3.3MB 9.6MB/s 
    100% |████████████████████████████████| 12.2MB 2.9MB/s 
spacy 2.0.16 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
  Found existing installation: setuptools 40.5.0
    Uninstalling setuptools-40.5.0:
      Successfully uninstalled setuptools-40.5.0
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
  Found existing installation: tensorboard 1.12.0
    Uninstalling tensorboard-1.12.0:
      Successfully uninstalled tensorboard-1.12.0
  Found existing installation: tensorflow 1.12.0
    Uninstalling tensorflow-1.12.0:
      Successfully uninstalled tensorflow-1.12.0


In [3]:
!pip install nltk

In [4]:
!pip install pyvi
!pip install glove_python

    100% |████████████████████████████████| 5.3MB 5.5MB/s 
    100% |████████████████████████████████| 51kB 19.3MB/s 
    100% |████████████████████████████████| 757kB 20.4MB/s 
  Running setup.py bdist_wheel for pyvi ... - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/b8/cb/53/54834bcc54e39de0e83897064a70335902e15ac84b44125b1a
  Running setup.py bdist_wheel for tabulate ... - done
  Stored in directory: /root/.cache/pip/wheels/2a/85/33/2f6da85d5f10614cbe5a625eab3b3aebfdf43e7b857f25f829
Successfully built pyvi tabulate
    100% |████████████████████████████████| 266kB 3.9MB/s 
  Running setup.py bdist_wheel for glove-python ... - \ | / - \ | done
  Stored in directory: /root/.cache/pip/wheels/88/4b/6d/10c0d2ad32c9d9d68beec9694a6f0b6e83ab1662a90a089a4b
Successfully built glove-python


In [0]:
import tensorflow as tf


In [0]:
import pandas as pd
import tensorflow as tf
from glove import Glove ,Corpus
import numpy as np
from collections import Counter
import os
import time
import re
import nltk
from pyvi import ViTokenizer, ViPosTagger

In [0]:
import Summarizer

In [0]:
import summarizer_data_utils
import summarizer_model_utils

In [0]:

giaitri = pd.read_csv('drive/My Drive/new-textsummary/data/GiaiTriThoiTrang_GD02.csv', encoding='utf-8')
giaoduc = pd.read_csv('drive/My Drive/new-textsummary/data/GiaoDuc_GD02.csv', encoding='utf-8')
kinhdoanh = pd.read_csv('drive/My Drive/new-textsummary/data/KinhDoanh_GD02.csv', encoding='utf-8')
suckhoe = pd.read_csv('drive/My Drive/new-textsummary/data/SucKhoe_GD02.csv',encoding='utf-8')
thethao = pd.read_csv('drive/My Drive/new-textsummary/data/TheThao_GD02.csv',encoding='utf-8')


In [0]:
giaitri=giaitri[['content','sumary']]
giaoduc = giaoduc[['content','sumary']]
kinhdoanh= kinhdoanh[['content','sumary']]
suckhoe = suckhoe[['content','sumary']]
thethao = thethao[['content','sumary']]


In [0]:
data = pd.concat([giaitri,giaoduc,kinhdoanh,suckhoe,thethao])

In [12]:
data.head()

,content,sumary
0,Tôi biết đến Angelina Jolie qua nhân vật Lara ...,Chị đã truyền cho con cái tình yêu thương.Và m...
1,"Trước khi phim công chiếu chính thức, tôi đã đ...","Không có một tình yêu hoàn hảo, không có một h..."
2,"Khác với Frida, cô có chất giọng cao, trong và...","Agnetha, sinh năm 1950, người sở hữu mái tóc v..."
3,"Bạn thân mến, bạn từng lên miền Tây Bắc Việt B...","Không cần thủ pháp âm nhạc cao siêu, bài hát v..."
4,Hồ Ngọc Hà xuất hiện sau cuối và đốt cháy sân ...,"Cuối liveshow thứ hai của ""Nhân tố bí ẩn"" tối ..."


In [13]:
len(data)

89066

In [14]:
data =data.dropna()
data.isnull().sum()

content    0
sumary     0
dtype: int64

In [0]:
data.rename(index = str, columns = {'content':'Text', 'sumary':'Summary'}, inplace = True)
data = data[['Summary', 'Text']]

In [16]:
data.head()

,Summary,Text
0,Chị đã truyền cho con cái tình yêu thương.Và m...,Tôi biết đến Angelina Jolie qua nhân vật Lara ...
1,"Không có một tình yêu hoàn hảo, không có một h...","Trước khi phim công chiếu chính thức, tôi đã đ..."
2,"Agnetha, sinh năm 1950, người sở hữu mái tóc v...","Khác với Frida, cô có chất giọng cao, trong và..."
3,"Không cần thủ pháp âm nhạc cao siêu, bài hát v...","Bạn thân mến, bạn từng lên miền Tây Bắc Việt B..."
4,"Cuối liveshow thứ hai của ""Nhân tố bí ẩn"" tối ...",Hồ Ngọc Hà xuất hiện sau cuối và đốt cháy sân ...



## Shuffle Data and Split small data



In [0]:
from sklearn.utils import shuffle
df = shuffle(data)
df = df.reset_index(drop=True)
len_data_train = int(len(df)*0.5)
train_data = df[:len_data_train]


In [18]:
len(train_data)

44533

## Process Data

In [0]:
texts_unprocessed = train_data.Text
summaries_unprocessed = train_data.Summary

In [0]:
stopwords  = pd.read_csv(r"vietnamese-stopwords.txt",header=None)

In [0]:
sumword = [i for i in stopwords[0]]

In [22]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [23]:
processed_texts, processed_summaries, word_counts = summarizer_data_utils.preprocess_texts_and_summaries(
    texts_unprocessed,
    summaries_unprocessed,
    sumword
    )

Processing Time:  388.7077944278717


In [24]:
len(processed_texts), len(processed_summaries)

(44533, 44533)

In [0]:
processed_summaries1 = []
processed_texts1 = []
for ind, x in enumerate(processed_texts):
  if len(x) <=120:
    processed_texts1.append(x)
    processed_summaries1.append(processed_summaries[ind])
    

In [26]:
len(processed_summaries1),len(processed_texts1)
  
  

(44097, 44097)

## Train Glove

In [27]:
data123 = processed_texts1 + processed_summaries1
len(data123)

88194

In [0]:
from glove import Corpus, Glove

In [30]:
corpus = Corpus()
corpus.fit(data123, window=10)
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)
glove.save('glove.model')

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


In [31]:
len(glove.dictionary)

116380

In [32]:
glove.most_similar("xinh đẹp",10)

[('quyến rũ', 0.7742126485090456),
 ('điển trai', 0.7225246539295245),
 ('ngoại hình', 0.7221268355970196),
 ('trẻ trung', 0.6992377525455419),
 ('gái', 0.6726056792367586),
 ('dịu dàng', 0.6721145191786568),
 ('giải đơn', 0.64333517125483),
 ('tôn vẻ', 0.6345025618550346),
 ('đẹp', 0.6319834149878312)]

## Create lookup dicts

In [33]:
specials = ["<EOS>", "<SOS>","<PAD>","<UNK>"]
word2ind, ind2word,  missing_words = summarizer_data_utils.create_word_inds_dicts(word_counts,
                                                                                  specials = specials,
                                                                                  min_occurences = 0)
print(len(word2ind), len(ind2word), len(missing_words))

117952 117952 0


## Pretrained embeddings


In [0]:
glove = np.load('glove.model')

In [0]:
embedding_dim = 100
nb_words = len(word2ind)
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
embeddings_index = glove['dictionary']
word_vectors =glove['word_vectors'] 
for word, i in word2ind.items():
    if word in embeddings_index:
        word_embedding_matrix[i] =word_vectors[embeddings_index[word]]
    else:
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

## Convert text and summaries

In [0]:
converted_texts, unknown_words_in_texts = summarizer_data_utils.convert_to_inds(processed_texts1,
                                                                                word2ind,
                                                                                eos = False)

In [0]:
converted_summaries, unknown_words_in_summaries = summarizer_data_utils.convert_to_inds(processed_summaries1,
                                                                                        word2ind,
                                                                                        eos = True,
                                                                                        sos = True)

## Split Data: Train, Test, Val

In [0]:
 len_train = int(len(converted_texts)*0.9)

In [0]:
data_train_texts, data_train_summary = converted_texts[:len_train] , converted_summaries[:len_train]

In [40]:
len(data_train_texts),len(data_train_summary)

(39687, 39687)

In [0]:
data_test_texts, data_test_summary = converted_texts[len_train:int(len(converted_texts)*0.95)],converted_summaries[len_train:int(len(converted_texts)*0.95)]

In [40]:
len(data_test_texts),len(data_test_summary)

(2204, 2204)

In [0]:
data_val_texts, data_val_summary = converted_texts[int(len(converted_texts)*0.95):],converted_summaries[int(len(converted_texts)*0.95):]

In [43]:
len(data_val_texts),len(data_val_summary)

(2205, 2205)

## Add HyperParamter

In [0]:
num_layers_encoder = 4
num_layers_decoder = 4
rnn_size_encoder = 256
rnn_size_decoder = 256

batch_size = 64
epochs = 0
clip = 5
keep_probability = 0.8
learning_rate = 0.05
max_lr=0.005
learning_rate_decay_steps = 100
learning_rate_decay = 0.90


pretrained_embeddings_path = word_embedding_matrix
summary_dir = os.path.join('./tensorboard/headlines')

use_cyclic_lr = True
inference_targets=True

## Train Model

In [1]:
summarizer_model_utils.reset_graph()
summarizer = Summarizer.Summarizer(word2ind,
                                   ind2word,
                                   save_path='drive/My Drive/new-textsummary/headlines/my_model',
                                   mode='TRAIN',
                                   num_layers_encoder = num_layers_encoder,
                                   num_layers_decoder = num_layers_decoder,
                                   rnn_size_encoder = rnn_size_encoder,
                                   rnn_size_decoder = rnn_size_decoder,
                                   batch_size = batch_size,
                                   clip = clip,
                                   keep_probability = keep_probability,
                                   learning_rate = learning_rate,
                                   max_lr=max_lr,
                                   learning_rate_decay_steps = learning_rate_decay_steps,
                                   learning_rate_decay = learning_rate_decay,
                                   epochs = epochs,
                                   pretrained_embeddings_path = pretrained_embeddings_path,
                                   use_cyclic_lr = use_cyclic_lr,)
#                                    summary_dir = summary_dir)           

summarizer.build_graph()
summarizer.train(data_train_texts, 
                 data_train_summary,
                 validation_inputs=data_val_texts,
                 validation_targets=data_val_summary
                 #restore_path = 'drive/My Drive/new-textsummary/headlines/my_model'
                )

NameError: ignored